In [ ]:
import bs4
#bs4 = pip install beautifulsoup
import urllib.request
import re
import nltk
import heapq
from fake_useragent import UserAgent
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
ua = UserAgent()
url = 'https://medium.com/@linda.xie/a-beginners-guide-to-monero-7a5df2c50ed9'

# Get data
user_agent = ua.google
request = urllib.request.Request(url)
request.add_header('User-Agent', user_agent)
get_data = urllib.request.urlopen(request)
post = get_data.read()

In [ ]:
parsed_post = bs4.BeautifulSoup(post, 'lxml')
para = parsed_post.find_all('p')

post_text = ""

In [ ]:
for p in para:
    post_text += p.text

In [ ]:
#Preprocessing
post_text = re.sub(r'\[[0-9]*\]',' ', post_text)
post_text = re.sub(r'\s+', ' ', post_text)

In [ ]:
form_post_text = re.sub('[^a-zA-Z]', ' ', post_text)
form_post_text = re.sub(r'\s+', ' ', form_post_text)

In [ ]:
#Converting txt to sentences
sentence_list = nltk.sent_tokenize(post_text)

In [ ]:
#Finding frequencies
stopwords = nltk.corpus.stopwords.words('english')

word_freq = {}
for word in nltk.word_tokenize(form_post_text):
    if word not in stopwords:
        if word not in word_freq.keys():
            word_freq[word] = 1
        else:
            word_freq[word] += 1

In [ ]:
max_freq = max(word_freq.values())

for word in word_freq.keys():
    word_freq[word] = (word_freq[word]/max_freq)

In [ ]:
#Calculating sentence scores
sentence_scores = {}
for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_freq.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_freq[word]
                else:
                    sentence_scores[sent] += word_freq[word]

In [ ]:
#Summary with heapg
#nlargest = top 5 sentences with highest scores
summary_sentences = heapq.nlargest(5, sentence_scores, key=sentence_scores.get)

summary = ' '.join(summary_sentences)
print(summary)